# Test Migration Plan

Test Migration plan class (in classes folder) by:
- First make ILP problem identical to last time
- Make mig class based on users created
- Try to devise a way to take ILP decision values and make plan out of that

In [1]:
# Import Generic Classes
import numpy as np
import copy

# Import All Custom Classes
import os, sys
sys.path.append(os.path.pardir+"/classes")

from Server import *
from User import *
from Link import *
from Job import *

# Import Solver Classes
from Optim_PlanGenerator import *
from SeqGreedy_PlanGenerator import*

In [2]:
import numpy as np
import math

class Migration_Plans:
    """
    Migration_Plans: 
        - Collects all migration plans generated for an entire system
        - mig_plan_dict : 6 x time_steps np array with rows
            - timeslot, user_active_flag, usr_voronoi, source_svr, dest_svr, mig_rate
    """
    
    def __init__(self, users, jobs, sim_params):
        """
        users - list of user class (used to extract user location)
        time_steps - how many timesteps to simulate user movement for
        """
        
        self.mig_plan_dict = {}
        self.sim_params = sim_params
        
        # Initialize dictionary 
        self.dict_init(users,jobs,sim_params.time_steps)
    
    """
    Init Helper Function
    """
    
    def dict_init(self, users, jobs, time_steps):
        
        param_collection = ["time_slot", "user_active_flag", 
                            "user_voronoi", "source_server", 
                            "dest_server", "mig_rate",
                            "mig_link_id", "service_link_id",
                            "service_thruput", "latency"]
        
        for u in range(len(users)):
            temp_usr_dict = {}
            
            for p in param_collection:
                temp_usr_dict[p] = np.zeros(time_steps)
            
            # Record active time and user voronoi
            for t in range(int(time_steps)):
                temp_usr_dict["time_slot"][t] = t
                temp_usr_dict["user_voronoi"][t] = users[u].user_voronoi_true[t]
                temp_usr_dict["user_active_flag"][t] = jobs[u].active_time[t]
            
            self.mig_plan_dict[u] = temp_usr_dict
    
    """
    Extraction Functions
    """
    
    def from_ILP(self, ILP_prob):
        """
        From decision variables h we want to replace the zero vectors of 
        self.mig_plan_dict with relevant values based on the decision vars
        
        Input: ILP_prob - an Optim_PlanGenerator object that already has been optimized
        """
        
        # 1. Loop through all h_vars and obtain those that have been selected
        h_hit_keys = []
        
        for h_key in ILP_prob.h_vars.keys():
            if ILP_prob.h_vars[h_key].varValue>0:
                h_hit_keys += [h_key]
            
        
        # 2. Loop through each of the users and isolate variables that pertain to them
        for j in range(len(ILP_prob.jobs)):
            uh_keys = []
            
            # Collect all keys from a certain user
            for hit_key in h_hit_keys:
                if hit_key[0] == j: # If of the correct user
                    uh_keys += [hit_key]
            
            
            # Reorder the hit keys in terms of time
            uh_keys_ordered = []
            curr_time = -1 
            for l in range(len(uh_keys)):
                time_key = None
                for key in uh_keys:
                    if key[3] == curr_time:
                        break
                uh_keys_ordered += [key]
                curr_time = key[4]
                uh_keys.remove(key)
        
            self.ILP_plan_extract(uh_keys_ordered,j)
                    
        
        return
    
    def from_seq_greedy(self,SG_prob):
        """
        From the migration plan problem, do the entire procedure of reserving resources and 
        generating final migration plan -- how will we do this for batch?
        """
        
        # Should add convert node informations

        self.SG_prob = SG_prob
        
        # Loop through time and user to generate incoming plans
        for t in range(self.sim_params.time_steps):
            for j in range(len(self.jobs)):
                            
                # 0. Update user server prob
                self.users[j].update_voronoi_probs(time_passed = t)
                
                # 1. Check for user arrival time
                if t == self.users[j].arrival_time:
                    
                    node_bans = []
                    path_bans = []
                    approved_flag = False
                    
                    while_idx = 0
                    
                    while not approved_flag:
                        print("usr:",j,"reserve:",while_idx)
                        while_idx += 1
                    
                        # 2. If user arrives, make plan
                        SG_prob.calc_all_costs(j=0)
                        SG_prob.obtain_minimum_cost_j(j=0)

                        # Start Node and End Node of Mig plan
                        start_node = self.convert_st2node[(-1,-1)]
                        end_node = self.convert_st2node[(-1,self.sim_params.time_steps)]

                        node_num, link_num = self.SG_prob.dijkstra_j(j=j,start_node=start_node,
                                                                     end_node=end_node)
                        
                        
                        # 3. Repeat resource reservation until no conflicts --> or reject job
                        node_bans, path_bans, approved_flag = self.SG_prob.check_reserve_resource(j,
                                                              node_num,link_num)
                        
                        # Update cost graph
                        if not approved_flag:
                            self.SG_prob.update_costs(j, node_bans,path_bans)
                    
                    
                    # Extract plan and record to system
                    self.seq_greedy_plan_extract(node_orders=node_num, 
                                                 link_path_orders=link_num, 
                                                 job_num=j)

    
    """
    Extraction function helpers
    """
    
    # ILP
    
    def ILP_plan_extract(self, uh_keys_ordered, job_num):
        """
        Loop through the ordered selected keys for a single user 
        And generate np arrays that will correspond to plans
        Inputs:
            uh_keys_ordered: list of tupels that represent h-variables in ILP Solution
            job_num: job id 
        """
        
        active = True
        inactive = False
        
        # Loop through each of the keys (Use switch cases below)
        for uh_key in uh_keys_ordered:
            start_time, end_time = uh_key[3], uh_key[4]
            source_server,dest_server = uh_key[1], uh_key[2]
            link_path = uh_key[5]
            
            case = (source_server >= 0, dest_server >= 0) # server source, dest active/inactive
            
            if case == (active, active) or case == (inactive,inactive):
                self.mig_plan_dict[job_num]["source_server"][start_time:end_time] = source_server
                self.mig_plan_dict[job_num]["dest_server"][start_time:end_time] = dest_server
                
                # Migration length find
                if source_server != dest_server:
                    mig_length = end_time - start_time
                    self.mig_plan_dict[job_num]["mig_rate"][start_time:end_time] = 1/mig_length
                    self.mig_plan_dict[job_num]["mig_link_id"][start_time:end_time] = link_path
                    
            elif case == (inactive, active) or case == (active, inactive):
                # The entire column in the plan is considered inactive/active
                self.mig_plan_dict[job_num]["source_server"][start_time:end_time] = source_server
                self.mig_plan_dict[job_num]["dest_server"][start_time:end_time] = source_server
    
    # HEuristic
    
    def seq_greedy_plan_extract(self, node_orders, link_path_orders, job_num):
        """
        Loop through the ordered selected nodes for a single user 
        And generate np arrays that will correspond to plans
        Inputs:
            node_orders: Sequence of nodes in mig graph that make up a plan
            link_path_orders : which path was taken between two nodes in mig graph
            job_num: job id 
        """
        
        active = True
        inactive = False
        
        # Pull pairs of nodes that are connected together
        pair_list = []
        path_idx = 0
        for i in range(len(node_orders)-1):
            pair_list += [(node_orders[i],node_orders[i+1])]
        
        # Loop through each of the keys (Use switch cases below)
        for (node1,node2) in pair_list:
            (source_server, start_time) = self.SG_prob.convert_node2st[job_num][node1]
            (dest_server, end_time) = self.SG_prob.convert_node2st[job_num][node2]
            link_path = link_path_orders[path_idx]
            path_idx += 1
            
            case = (source_server >= 0, dest_server >= 0) # server source, dest active/inactive
            
            if case == (active, active) or case == (inactive,inactive):
                self.mig_plan_dict[job_num]["source_server"][start_time:end_time] = source_server
                self.mig_plan_dict[job_num]["dest_server"][start_time:end_time] = dest_server
                
                # Migration length find
                if source_server != dest_server:
                    mig_length = end_time - start_time
                    self.mig_plan_dict[job_num]["mig_rate"][start_time:end_time] = 1/mig_length
                    self.mig_plan_dict[job_num]["mig_link_id"][start_time:end_time] = link_path
                    
            elif case == (inactive, active) or case == (active, inactive):
                # The entire column in the plan is considered inactive/active
                self.mig_plan_dict[job_num]["source_server"][start_time:end_time] = source_server
                self.mig_plan_dict[job_num]["dest_server"][start_time:end_time] = source_server
    
    # General
        
    def reserve_service_bw(self,links,jobs):
        """
        take into a consideration the resources at each link at each timestep, and determine
        Inputs:
        links - a link instance of the simulation
        jobs - a list of job objects each with their job size 
        
        Updates migration plan to determine throughput of service at each instance
        """
        
        # Loop through each ts
        
        # Loop through each plan
        
        # 1. Select link randomly from available options
        
        # 2. If source/dest differ + active 
        
        # a. Loop through each of the links that is for this job
        
        # b. Loop through each of the active jobs that use this link
        
        # c. Find the bottleneck thruput based on proportions and reserve
        # we will have slight inefficiency due to sequential reserve system but it'll be redundant
        # across all users
        
        return

### Make Users, Servers, Jobs

In [3]:
"""
Make Simulation Parameters
"""
sim_param = Sim_Params(time_steps=5, x_length = 5, y_length = 5, max_edge_length=2)
boundaries = np.array([[0,sim_param.x_length],[0,sim_param.y_length]])


"""
Make Job Profiles
"""
# REsources used are CPU (no. cores) storage (GB), and RAM (GB)
# througput is in mb/s
# Latency is in ms

job_profile1 = Job_Profile(job_name = "VR",
                           latency_req_range=[0, 0], 
                           thruput_req_range=[50/1000, 200/1000], 
                           length_range=[5,5],  
                           placement_rsrc_range = np.array([[2,3],[8,16],[2,5]]),
                           migration_amt_range = [5, 10],
                           latency_penalty_range = [1,11])#[0.05, 0.1]) 

job_profile2 = Job_Profile(job_name = "Assistant",
                           latency_req_range=[100, 200],
                           thruput_req_range=[5/1000, 20/1000],
                           length_range=[1,5],
                           placement_rsrc_range = np.array([[1,1],[0.5,1],[0.5,1]]),
                           migration_amt_range = [0.5, 1],
                           latency_penalty_range = [0.01, 0.05])

job_profile3 = Job_Profile(job_name = "AR",
                           latency_req_range=[50, 80], 
                           thruput_req_range=[20/1000, 50/1000],
                           length_range=[1,5],
                           placement_rsrc_range = np.array([[1,2],[2,4],[1,2]]),
                           migration_amt_range = [2, 3],
                           latency_penalty_range = [0.03, 0.08])

job_profiles = [job_profile1, job_profile2, job_profile3]


"""
Make Servers
"""

# Server Settings
num_server_l1 = 3
num_server_l2 = 2
num_server_l3 = 1

num_resource = 3
weak_range = np.array([[4,8],[1000,1500],[4,16]])
strong_range = np.array([[50,100],[100000,150000],[300,600]])

rsrc_cost = np.array([0.03, 0.01, 0.05])

# Generate Server
servers_l1 = []
servers_l2 = []
servers_l3 = []
idx_counter = 0

for i in range(num_server_l1):
    servers_l1.append(Server(boundaries,level=1,rand_locs=True,locs=None))
    servers_l1[-1].server_resources(num_resource, weak_range, strong_range)
    servers_l1[-1].assign_id(idx_counter)
    servers_l1[-1].server_resources_cost(num_resource,rsrc_cost)
    idx_counter += 1
    
for i in range(num_server_l2):
    servers_l2.append(Server(boundaries,level=2,rand_locs=True,locs=None))
    servers_l2[-1].server_resources(num_resource, weak_range, strong_range)
    servers_l2[-1].assign_id(idx_counter)
    servers_l2[-1].server_resources_cost(num_resource,rsrc_cost)
    idx_counter += 1
    
for i in range(num_server_l3):
    servers_l3.append(Server(boundaries,level=3,rand_locs=False,locs=np.array([200,200])))
    servers_l3[-1].server_resources(num_resource, weak_range, strong_range)
    servers_l3[-1].assign_id(idx_counter)
    servers_l3[-1].server_resources_cost(num_resource,rsrc_cost)
    idx_counter += 1
    
servers = servers_l1 + servers_l2 + servers_l3


"""
Make Links
"""

# Link Settings
num_link = [0,1,2]
prob_link = [0,1,0]
lv_minmax = np.array(([[500,1000],[10000,20000],[30000,50000]]))
lv1_transmission = 1
link_costs = np.array([0.05, 0.02, 0.01])
latency_settings = [1000, 100] #[ms per switch, ms per mile]

links = Link(servers, num_link, prob_link, lv_minmax, link_costs, latency_settings,lv1_transmission)


"""
Make Users
"""

# User Settings
num_user_m0 = 0 # Pedestrian
num_user_m1 = 1 # Public Transport
num_user_m2 = 0 # Vehicle

max_speed = 2.5
lamdas = [1/0.25,1/0.83,1/1.67] # 3 mph, 10 mph, 20 mph
num_path = 10

# Generate Server
users_m0 = []
users_m1 = []
users_m2 = []
idx_counter = 0

for i in range(num_user_m0):
    users_m0 += [User(boundaries, sim_param.time_steps, 0, lamdas, max_speed, num_path)]
    users_m0[-1].generate_MC(servers)
    users_m0[-1].assign_id(idx_counter)
    idx_counter += 1
    
for i in range(num_user_m1):
    users_m1 += [User(boundaries, sim_param.time_steps, 1, lamdas, max_speed, 1)]
    users_m1[-1].generate_MC(servers)
    users_m1[-1].assign_id(idx_counter)
    idx_counter += 1

for i in range(num_user_m2):
    users_m2 += [User(boundaries, sim_param.time_steps, 2, lamdas, max_speed, num_path)]
    users_m2[-1].generate_MC(servers)
    users_m2[-1].assign_id(idx_counter)
    idx_counter += 1

users = users_m0 + users_m1 + users_m2
    
    
"""
Make Jobs
- "I'm just going to do it"
"""

# Job settings
job_type0 = 1 # VR
job_type1 = 0 # Assistant
job_type2 = 0 # AR

jobs0 = []
jobs1 = []
jobs2 = []
idx_counter = 0

total_job_count = job_type0+job_type1+job_type2
draw_job_id = np.random.choice(total_job_count, total_job_count, replace=False)

for i in range(job_type0):
    jobs0 += [Job(job_type = 0,
                  user_id = draw_job_id[idx_counter],
                  time_steps=sim_param.time_steps,
                  job_profiles = job_profiles)]
    idx_counter += 1
    
for i in range(job_type1):
    jobs1 += [Job(job_type = 1,
                  user_id = draw_job_id[idx_counter],
                  time_steps=sim_param.time_steps,
                  job_profiles = job_profiles)]
    idx_counter += 1
    
for i in range(job_type2):
    jobs2 += [Job(job_type = 2,
                  user_id = draw_job_id[idx_counter],
                  time_steps=sim_param.time_steps,
                  job_profiles=job_profiles)]
    idx_counter += 1
    
jobs = jobs0 + jobs1 + jobs2

In [4]:
users[0].server_prob

array([[1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 1.],
       [0., 0., 0., 0., 0.],
       [0., 1., 1., 0., 0.],
       [0., 0., 0., 0., 0.]])

### Make ILP Solver

In [5]:
super_prob = PlanGenerator(users, servers, links, jobs, sim_param)
optim_prob = Optim_PlanGenerator(users, servers, links, jobs, sim_param)

In [6]:
optim_prob.prob.solve()
print("Status:", constants.LpStatus[optim_prob.prob.status])

Status: Optimal


### Migration Plan Class

In [7]:
ILP_mig_plan = Migration_Plans(users, jobs, sim_param) 
ILP_mig_plan.from_ILP(optim_prob)

In [8]:
ILP_mig_plan.mig_plan_dict

{0: {'time_slot': array([0., 1., 2., 3., 4.]),
  'user_active_flag': array([1., 1., 1., 1., 1.]),
  'user_voronoi': array([0., 4., 4., 2., 2.]),
  'source_server': array([0., 4., 4., 2., 2.]),
  'dest_server': array([4., 4., 2., 2., 2.]),
  'mig_rate': array([1., 0., 1., 0., 0.]),
  'mig_link_id': array([0., 0., 1., 0., 0.]),
  'service_link_id': array([0., 0., 0., 0., 0.]),
  'service_thruput': array([0., 0., 0., 0., 0.]),
  'latency': array([0., 0., 0., 0., 0.])}}

In [9]:
for v in optim_prob.prob.variables():
    if v.varValue>0:
        print(v.name, "=", v.varValue)

h_(0,_0,_4,_0,_1,_0) = 1.0
h_(0,_2,_2,_3,_4,_0) = 1.0
h_(0,_2,__1,_4,_5,_0) = 1.0
h_(0,_4,_2,_2,_3,_1) = 1.0
h_(0,_4,_4,_1,_2,_0) = 1.0
h_(0,__1,_0,__1,_0,_0) = 1.0


### Draw Plan From Seq Greedy

In [10]:
SG_prob = SeqGreedy_PlanGenerator(users, servers, links, jobs, sim_param)
SG_prob.calc_all_costs(j=0)
SG_prob.obtain_minimum_cost_j(j=0)

# Start Node and End Node of Mig plan
start_node = SG_prob.convert_st2node[0][(-1,-1)]
end_node = SG_prob.convert_st2node[0][(-1,SG_prob.sim_params.time_steps)]

In [11]:
node_num, link_num = SG_prob.dijkstra_j(j=0, start_node = start_node, end_node = end_node)

In [12]:
node_num

[0, 2, 13, 20, 25, 32, 36]

In [13]:
SG_mig_plan = Migration_Plans(users, jobs, sim_param)
SG_mig_plan.SG_prob = SG_prob
SG_mig_plan.seq_greedy_plan_extract(node_orders=node_num, link_path_orders=link_num, job_num=0)

In [14]:
SG_prob.resource_constraints.server_rsrc[:,:,0]

array([[7.94153083e+00, 1.14067042e+03, 6.56762028e+00],
       [5.03675820e+00, 1.01503772e+03, 1.36960528e+01],
       [6.44770432e+00, 1.33675193e+03, 1.39087579e+01],
       [6.67380103e+01, 1.20114954e+05, 4.03088525e+02],
       [8.85468431e+01, 1.04055924e+05, 5.05298907e+02],
       [1.00000000e+09, 1.00000000e+09, 1.00000000e+09]])

In [16]:
node_bans, path_bans, approved_flag = SG_prob.check_reserve_resource(j=0,
                                                                     shortest_path=node_num,
                                                                     shortest_path_link_idx = link_num)

In [20]:
SG_prob.resource_constraints.server_rsrc[:,:,0]

array([[5.18894941e+00, 1.12699603e+03, 2.65348770e+00],
       [5.03675820e+00, 1.01503772e+03, 1.36960528e+01],
       [6.44770432e+00, 1.33675193e+03, 1.39087579e+01],
       [6.67380103e+01, 1.20114954e+05, 4.03088525e+02],
       [8.57942617e+01, 1.04042250e+05, 5.01384775e+02],
       [1.00000000e+09, 1.00000000e+09, 1.00000000e+09]])

In [18]:
path_bans

[]

In [19]:
approved_flag

True